In [6]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import recall_score

s3 = boto3.resource('s3')
bucket_name = 'grant-gonnerman-data-445'
bucket = s3.Bucket(bucket_name)

file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [7]:
# removing NA 
heart = heart.dropna()

In [8]:
# defining input and target variables 
x = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y = heart['TenYearCHD']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [9]:
# Ada Boost model
# building the model
ada_md = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(x_train, y_train)

# predicting on test
ada_pred = ada_md.predict_proba(x_test)[:,1]

#liklyhoods to labels
ada_label = np.where(ada_pred < .1, 0, 1)

# computing recall
print('recall of ada boost model:', recall_score(y_test, ada_label))

recall of ada boost model: 1.0


In [10]:
# RF model
# building the model 
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(x_train, y_train)

# predicting on test
rf_pred = rf_md.predict_proba(x_test)[:,1]

#liklyhoods to labels
rf_label = np.where(rf_pred < .1, 0, 1)

# computing recall
print('recall of ada boost model:', recall_score(y_test, rf_label))

recall of ada boost model: 0.875


In [ ]:
# based on the recall i would use the Ada Boost model because it has the higher recall. 